In [ ]:
 def explore_grid(n, matrix, num_aliens):                                                    # initialize function

  # Generate two random values within the range [1, n]
  random_value1 = random.randint(1, n-2)                                        # generate random y value
  random_value2 = random.randint(1, n-2)                                        # generate random x value

  starting_cell=matrix[random_value1][random_value2]                            # initialize open_cell
  # Open First Cell
  matrix[starting_cell['self'][0]][starting_cell['self'][1]]['value']=1         # Open First Cell
  current_open_cell=starting_cell                                               # initialize current_open_cell as open_cell
  possible_future_cells=[]                                                      # initialize possible_future_cells

  while True:
    if possible_future_cells:                                                 # if possible_future_cells not empty
      new_chosen_cell = random.choice(possible_future_cells)                  # initialize new_chosen_cell as random cell from possible_future_cells
      matrix[new_chosen_cell['self'][0]][new_chosen_cell['self'][1]]['value']=1 #open the new_choesn_cell
      current_open_cell=new_chosen_cell                                       # set current_open_cell to be the newest open cell
      possible_future_cells.remove(new_chosen_cell)                           # remove new_chosen_cell from possible_future_cells
    possible_future_cells = detect_possible_future_cells(current_open_cell, possible_future_cells, n) # add valid neighbors of current_open_cell to possible_future_cells
    if (starting_cell in possible_future_cells):
        possible_future_cells.remove(starting_cell)
    possible_future_cells = back_check_possible_future_cells(possible_future_cells) # remove invalid cells from possible_future_cells
    if (starting_cell in possible_future_cells):
        possible_future_cells.remove(starting_cell)

    if (len(possible_future_cells)==0):                                       # if no more possible_future_cells to add end while loop
      break

  dead_ends=find_dead_ends(matrix,n)
  matrix, des = open_half_of_dead_ends(matrix, dead_ends, n)

  aliens = initialize_aliens(num_aliens, matrix)
  for i in aliens:
    if matrix[i['self'][0]][i['self'][1]]['value']!='C':
      matrix[i['self'][0]][i['self'][1]]['value'] = 'A'

  crew_start_cell = random.choice(detect_open_cells(matrix))
  matrix[crew_start_cell[0]][crew_start_cell[1]]['value']= 'C'

  if (len(detect_open_neighbors_bot_1(matrix[crew_start_cell[0]][crew_start_cell[1]]))==0):
    modify_blocked_cells(matrix,crew_start_cell)

  return matrix, aliens, crew_start_cell                                                               # return completed matrix

In [ ]:
from collections import deque

class Cell:
    def __init__(self, x, y, value):
        self.x = x
        self.y = y
        self.value = value
        self.parent = None
        self.visited = False

def create_grid(rows, cols):
    grid = [[Cell(i, j, 'E') for j in range(cols)] for i in range(rows)]
    return grid

def find_bot_start(grid):
    for row in grid:
        for cell in row:
            if cell.value == 'B':
                return cell
    return None

def is_valid(cell, grid):
    return (
        0 <= cell.x < len(grid) and
        0 <= cell.y < len(grid[0]) and
        not grid[cell.x][cell.y].visited and
        grid[cell.x][cell.y].value != 'A'
    )

def get_neighbors(cell, grid):
    directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]  # Right, Down, Left, Up
    neighbors = []

    for dx, dy in directions:
        new_x, new_y = cell.x + dx, cell.y + dy
        if is_valid(Cell(new_x, new_y, ''), grid):
            neighbors.append(grid[new_x][new_y])

    return neighbors

def bfs(grid, start):
    queue = deque([start])
    start.visited = True

    while queue:
        current_cell = queue.popleft()

        if current_cell.value == 'C':  # If Crewmate found
            path = []
            while current_cell:
                path.append((current_cell.x, current_cell.y))
                current_cell = current_cell.parent
            return path[::-1]  # Return reversed path

        neighbors = get_neighbors(current_cell, grid)
        for neighbor in neighbors:
            neighbor.visited = True
            neighbor.parent = current_cell
            queue.append(neighbor)

    return None  # No path found

# Example usage
# grid = create_grid(5, 5)
# grid[0][0].value = 'B'  # Bot Start Position
# grid[3][3].value = 'C'  # Crewmate Position
# grid[2][2].value = 'A'  # Alien Position

# start_cell = find_bot_start(grid)
# if start_cell:
#     path_to_crewmate = bfs(grid, start_cell)
#     if path_to_crewmate:
#         print("Path to crewmate found:", path_to_crewmate)
#     else:
#         print("No path to crewmate found.")
# else:
#     print("Bot start position not found.")


In [ ]:
def initialize_ship(n):                                                         # initialize initialize_ship function
  rows = n                                                                      # initialize number of rows
  cols = n                                                                      # initialize number of columns

  # Create a matrix of dictionaries
  matrix = []                                                                   # initialize matrix

  # Fill the matrix with dictionaries
  for i in range(rows):
      row = []
      for j in range(cols):
          left=j-1
          right=j+1
          top=i-1
          bottom=i+1
          #Testing for valid neighbor coordinates
          if (top<0):
              top=-1
          elif (bottom==n):
              bottom=-1
          elif (left<0):
              left=-1
          elif (right==n):
              right=-1
          # Create a dictionary with some key-value pairs
          dictionary = {
              'value': 0,
              'self': [i,j],
              'left': [i,left],
              'right': [i,right],
              'top': [top,j],
              'bottom': [bottom,j],
              'visited':'false',
              'prev':[]
          }
          row.append(dictionary)
      matrix.append(row)
  return matrix, n

In [ ]:
def draw_ship(matrix):
  print('  ', end=' ')
  for i in range(n):
    print(i,end='  ')
  print("\n")
  j = 0
  for row in matrix:
    print(j, end=' ')
    j = j + 1
    for elem in row:
      if (elem['value']==1):
        print("o ",end=' ')
      elif (elem['value'] == 'C'):
        print("C ", end = ' ')
      elif (elem['value'] == 'A'):
        print("A ", end = ' ')
      elif (elem['value'] == 'C*'):
        print("c ", end = ' ')
      elif (elem['value'] == 'B'):
        print("B ", end = ' ')
      else:
        print("x ",end=' ')
    print(" ")

In [ ]:
import pandas as pd

def draw_ship_df(matrix):


  columns_123=[]
  rows_123=[]
  for i in range(len(matrix)):
    columns_123.append(i)
    rows_123.append(i)

  values_matrix=[]

  for i in range(len(matrix)):
    temp_row=[]
    for j in range(len(matrix)):
      elem=matrix[i][j]['value']
      edit_val=""
      if (elem==1):
        edit_val="o"
      elif (elem == 'C'):
        edit_val="C"
      elif (elem == 'A'):
        edit_val="A"
      elif (elem == 'C*'):
        edit_val="c"
      elif (elem == 'B'):
        edit_val="B"
      else:
        edit_val="x"
      temp_row.append(edit_val)
    values_matrix.append(temp_row)

  df = pd.DataFrame(values_matrix, columns=columns_123, index=rows_123)

  # for i in range(len(df)):
  #   for j in range(len(df)):
  #     elem=df.iloc[i][j]
  #     if (elem==1):
  #       df.iloc[i][j]="o"
  #     elif (elem == 'C'):
  #       df.iloc[i][j]="C"
  #     elif (elem == 'A'):
  #       df.iloc[i][j]="A"
  #     elif (elem == 'C*'):
  #       df.iloc[i][j]="c"
  #     elif (elem == 'B'):
  #       df.iloc[i][j]="B"
  #     else:
  #       df.iloc[i][j]="x"

  # print(df.head())
  print(df.to_string())
  print("\n")

In [ ]:
def detect_possible_future_cells(open_cell, possible_future_cells, n):
  arr=['left','right','top','bottom']
  for i in arr:

    counter=0

    if(open_cell[i][1]<0 or open_cell[i][1] == n):
      continue

    neighbor_cell = matrix[open_cell[i][0]][open_cell[i][1]]

    if (neighbor_cell['self'][0]==-1 or neighbor_cell['self'][1]==-1):
      continue

    open_neighbor_count=0

    for j in arr:
      if(neighbor_cell[j][1]<0 or neighbor_cell[j][1] == n):
        continue

      neighbor_cell_neighbor = matrix[neighbor_cell[j][0]][neighbor_cell[j][1]]

      if (neighbor_cell_neighbor['self'][0]==-1 or neighbor_cell_neighbor['self'][1]==-1):
        continue

      if (neighbor_cell_neighbor['value']==1):
        counter=counter+1

      if (neighbor_cell_neighbor['value']==1):
        open_neighbor_count=open_neighbor_count+1

    if (counter<2 and open_neighbor_count>=1):
      possible_future_cells.append(neighbor_cell)
  return possible_future_cells

In [ ]:
def back_check_possible_future_cells(possible_future_cells):
  arr=['left','right','top','bottom']
  for cell in possible_future_cells:
    counter=0
    for j in arr:
      if(cell[j][1]<0 or cell[j][1] == n):
        continue
      neighbor_cell = matrix[cell[j][0]][cell[j][1]]
      if (neighbor_cell['value']==1):
        counter=counter+1
    if (counter>1):
      possible_future_cells.remove(cell)
  return possible_future_cells

In [ ]:
def find_dead_ends(matrix, n):
  dead_ends=[]
  arr=['left','right','top','bottom']
  for row in matrix:
    for elem in row:
      if elem['value']==1:
        open_count=0
        for i in arr:
          if (elem[i][0]==-1 or elem[i][1]==-1 or elem[i][0]==n or elem[i][1]==n):
            continue;
          neighbor=matrix[elem[i][0]][elem[i][1]]
          if neighbor['value']==1:
            open_count=open_count+1
        if (open_count==1):
          dead_ends.append(elem)
  return dead_ends

In [ ]:

def print_dead_ends(matrix, dead_ends):
  print('  ', end=' ')
  for i in range(n):
    print(i,end='  ')
  print("\n")
  j = 0
  for row in matrix:
    print(j, end=' ')
    j = j + 1
    for elem in row:
      if (elem not in dead_ends):
        print('N',end=' ')
      else:
        print('D',end=' ')
    print(" ")

In [ ]:
def modify_blocked_cells(matrix,crew_cell):
  arr=['left','right','top','bottom']
  if len(detect_open_neighbors_bot_1(matrix[crew_cell[0]][crew_cell[1]]))==0:
    ch1=random.choice(arr)
    while(matrix[crew_cell[0]][crew_cell[1]][ch1][0]==-1 or matrix[crew_cell[0]][crew_cell[1]][ch1][1]==-1):
      arr.remove(ch1)
      ch1=random.choice(arr)
    x1=matrix[crew_cell[0]][crew_cell[1]][ch1][0]
    y1=matrix[crew_cell[0]][crew_cell[1]][ch1][1]
    matrix[x1][y1]['value']=1

In [ ]:
def open_half_of_dead_ends(matrix, dead_ends, n):
  num_dead_ends=len(dead_ends)
  nodes_to_open=int(num_dead_ends/2+1)
  for i in range(nodes_to_open):
    try:
      dead_end_to_open = random.choice(dead_ends)
    except:
      continue
    arr=['left','right','top','bottom']
    valid_neighbors_to_open=[]
    for i in arr:
      if (dead_end_to_open[i][0]==-1 or dead_end_to_open[i][1]==-1 or dead_end_to_open[i][0]==n or dead_end_to_open[i][1]==n):
        continue
      neighbor=matrix[dead_end_to_open[i][0]][dead_end_to_open[i][1]]
      if (neighbor['value']==1):
        continue
      elif (neighbor['self'][0]==-1 or neighbor['self'][1]==-1 or neighbor['self'][0]==n or neighbor['self'][1]==n):
        continue
      else:
        valid_neighbors_to_open.append(neighbor['self'])
    if len(valid_neighbors_to_open)!=0:
      dead_end_neighbor_to_open = random.choice(valid_neighbors_to_open)
    matrix[dead_end_neighbor_to_open[0]][dead_end_neighbor_to_open[1]]['value']=1
    dead_ends.remove(dead_end_to_open)
  return matrix, dead_ends

In [ ]:
def detect_open_neighbors_bot_1(current_cell):
  open_neighbors = []
  arr=['left','right','top','bottom']
  for i in arr:

    if (current_cell[i][0]==-1 or current_cell[i][1]==-1 or current_cell[i][0]==n or current_cell[i][1]==n):
        continue
    if matrix[current_cell[i][0]][current_cell[i][1]]['value']==0:
      continue
    else:
      open_neighbors.append(matrix[current_cell[i][0]][current_cell[i][1]]['self'])

  return open_neighbors

In [ ]:
def detect_open_neighbors(current_cell):
  open_neighbors = []
  arr=['left','right','top','bottom']
  for i in arr:

    if (current_cell[i][0]==-1 or current_cell[i][1]==-1 or current_cell[i][0]==n or current_cell[i][1]==n):
        continue
    if(matrix[current_cell[i][0]][current_cell[i][1]]['value'] == 1):
       open_neighbors.append(matrix[current_cell[i][0]][current_cell[i][1]]['self'])
    if(matrix[current_cell[i][0]][current_cell[i][1]]['value'] == 'C'):
      open_neighbors.append(matrix[current_cell[i][0]][current_cell[i][1]]['self'])
    if(matrix[current_cell[i][0]][current_cell[i][1]]['value'] == 'C*'):        # COUNTING C* AS OPEN!!!!!!!!!!!!!
      open_neighbors.append(matrix[current_cell[i][0]][current_cell[i][1]]['self'])
    if(matrix[current_cell[i][0]][current_cell[i][1]]['value'] == 'B'):
      open_neighbors.append(matrix[current_cell[i][0]][current_cell[i][1]]['self'])

  return open_neighbors

In [ ]:
#adding this because the above code counts an alien on a crew mate as open but we don't want multiple aliens in the same cell
def detect_open_neighbors_for_aliens(current_cell):
  open_neighbors = []
  arr=['left','right','top','bottom']
  for i in arr:

    if (current_cell[i][0]==-1 or current_cell[i][1]==-1 or current_cell[i][0]==n or current_cell[i][1]==n):
        continue
    if(matrix[current_cell[i][0]][current_cell[i][1]]['value'] == 1):
       open_neighbors.append(matrix[current_cell[i][0]][current_cell[i][1]]['self'])
    if(matrix[current_cell[i][0]][current_cell[i][1]]['value'] == 'C'):
      open_neighbors.append(matrix[current_cell[i][0]][current_cell[i][1]]['self'])
    if(matrix[current_cell[i][0]][current_cell[i][1]]['value'] == 'B'):
      open_neighbors.append(matrix[current_cell[i][0]][current_cell[i][1]]['self'])

  return open_neighbors

In [ ]:
def detect_open_cells(matrix):
  open_cells = []
  for i in range(len(matrix)):
    for j in range(len(matrix)):
      if (matrix[i][j]['value'] == 1 or 'C'):
        open_cells.append(matrix[i][j]['self'])
  return open_cells

In [ ]:
def initialize_aliens(n, matrix):
  aliens = []                                                # initialize initialize_ship function                                                            # initialize matrix
  current_alien_start_cells = []
  for i in range(n):
    open_cells = detect_open_cells(matrix)
    alien_start_cell = random.choice(open_cells)
    while alien_start_cell in current_alien_start_cells:                        #seems like a shit solution
      alien_start_cell = random.choice(open_cells)
    current_alien_start_cells.append(alien_start_cell)
    dictionary = {
      'alien_id' : i,
      'self': alien_start_cell,
      'open_neighbors' : detect_open_neighbors_for_aliens(matrix[alien_start_cell[0]][alien_start_cell[1]])
    }
    aliens.append(dictionary)
    matrix[alien_start_cell[0]][alien_start_cell[1]]['value'] = 'A'
  return aliens

In [ ]:
def take_step(matrix, aliens, crew_start_cell, bot_cell, time_steps):
  # print("From start of take_step: Bot position:", bot_cell, "Bot value", matrix[bot_cell[0]][bot_cell[1]]['value'])                           # VERY HELPFUL FOR DEBUG
  # print("From take_step: Crew position", crew_start_cell)                                                                                     # VERY HELPFUL FOR DEBUG
  import random
  import copy

  time_steps = time_steps + 1

  random.shuffle(aliens)
  for i in aliens:
    old_alien_location = i['self']
    # print("From take_step: Alien", i["alien_id"], "action: ")                                                                                 # VERY HELPFUL FOR DEBUG
    if (matrix[crew_start_cell[0]][crew_start_cell[1]]['value'] == 'B'):
      print("From take_step: Bot found crew memeber loading new crew memeber")                                                                # VERY HELPFUL FOR DEBUG
      crew_start_cell = random.choice(detect_open_cells(matrix))
      if (len(detect_open_neighbors_bot_1(matrix[crew_start_cell[0]][crew_start_cell[1]]))==0):
        modify_blocked_cells(matrix,crew_start_cell)
      print("Frome take_step: New Crew position: ", crew_start_cell)                                                                          # VERY HELPFUL FOR DEBUG
      matrix[crew_start_cell[0]][crew_start_cell[1]]['value'] = 'C'
    for j in aliens:
      if i['alien_id'] != j['alien_id']:
        if (matrix[i['self'][0]][i['self'][1]]['self'] == matrix[j['self'][0]][j['self'][1]]['self']):
          matrix[i['self'][0]][i['self'][1]]['value'] = 'A'
    if (matrix[i['self'][0]][i['self'][1]]['self'] == matrix[bot_cell[0]][bot_cell[1]]['self']):                #Bot is moving into an alien cell at the same time as the alien moves? not sure if this is correct
      matrix[i['self'][0]][i['self'][1]]['value'] = 'B'
    if (matrix[i['self'][0]][i['self'][1]]['self'] == matrix[crew_start_cell[0]][crew_start_cell[1]]['self']):
      matrix[i['self'][0]][i['self'][1]]['value'] = 'C'
    else:
      matrix[i['self'][0]][i['self'][1]]['value'] = 1
    i['open_neighbors'] = detect_open_neighbors_for_aliens(matrix[i['self'][0]][i['self'][1]])
    try:
      #print("From take_step: Current Alien location: ", i['self'])                                                                             # VERY HELPFUL FOR DEBUG
      i['self'] = random.choice(i['open_neighbors'])
      #print("From take_step: New Alien location: ", i['self'])                                                                                 # VERY HELPFUL FOR DEBUG
    except:
      matrix[i['self'][0]][i['self'][1]]['value'] = 'A'
      # print("From take_step: except:")                                                                                                        # VERY HELPFUL FOR DEBUG
      # print("From take_step: Ship after alien", i["alien_id"], "moves")                                                                       # VERY HELPFUL FOR DEBUG
      # draw_ship(matrix)                                                                                                                       # VERY HELPFUL FOR DEBUG
      continue
    if matrix[i['self'][0]][i['self'][1]]['value']=='C' or matrix[i['self'][0]][i['self'][1]]['value']=='C*':
      matrix[i['self'][0]][i['self'][1]]['value'] = 'C*'
    else:
      matrix[i['self'][0]][i['self'][1]]['value'] = 'A'
    # print("From take_step: Ship after alien", i["alien_id"], "move")                                                                          # VERY HELPFUL FOR DEBUG
    # print("From take_step: ", i["alien_id"], "moves from", old_alien_location, "to ", i['self'])                                              # VERY HELPFUL FOR DEBUG
    # print("From take_step: New cell value", matrix[i['self'][0]][i['self'][1]]['value'])                                                      # VERY HELPFUL FOR DEBUG
    # draw_ship(matrix)                                                                                                                         # VERY HELPFUL FOR DEBUG
    # print("----------------------")                                                                                                           # VERY HELPFUL FOR DEBUG

  # print("From end of take_step: Bot position:", bot_cell, "Bot value", matrix[bot_cell[0]][bot_cell[1]]['value'])                             # VERY HELPFUL FOR DEBUG
    if matrix[bot_cell[0]][bot_cell[1]]['value']=='A' or matrix[bot_cell[0]][bot_cell[1]]['value']=='C*':                                         # VERY HELPFUL FOR DEBUG
      print("From take_step: take_step has detected the bot being caught by an alien: Bot location: ", bot_cell, "Alien cell: ", i)             # VERY HELPFUL FOR DEBUG
      return "The bot was caught by the aliens!", crew_start_cell, time_steps

  # draw_ship(matrix)
  draw_ship_df(matrix)
  # print("----------------------------------------------------------------------------------")                                                 # VERY HELPFUL FOR DEBUG

  return "Status normal", crew_start_cell, time_steps

In [ ]:
#redundant?
def detect_alien_free_neighbors(current_cell,aliens):
  open_n=detect_open_neighbors(current_cell)
  for alien in aliens:
    if alien['self'] in open_n:
      open_n.remove(alien['self'])
  return open_n

In [ ]:
#redundant?
def detect_alien_free_cells(matrix,aliens):
  open_cells=detect_open_cells(matrix)
  for alien in aliens:
    if alien['self'] in open_cells:
      open_cells.remove(alien['self'])
  return open_cells

In [ ]:
def create_bot(matrix,aliens):
  alien_free_cells=detect_alien_free_cells(matrix,aliens)
  bot_start_cell=random.choice(alien_free_cells)
  matrix[bot_start_cell[0]][bot_start_cell[1]]['value']='B'
  return bot_start_cell

In [ ]:
def move_bot(bot_cell,path,aliens,crew_cell, time_steps):
  if matrix[bot_cell[0]][bot_cell[1]]['value']=='A' or matrix[bot_cell[0]][bot_cell[1]]['value']=='C*':
    return "The bot was caught by the aliens", bot_cell, crew_cell, time_steps
  for next in path[1::]:
    matrix[bot_cell[0]][bot_cell[1]]['value']=1
    matrix[next[0]][next[1]]['value']='B'
    bot_cell=next
    if matrix[bot_cell[0]][bot_cell[1]]['value']=='A' or matrix[bot_cell[0]][bot_cell[1]]['value']=='C*':
      return "The bot was caught by the aliens!", bot_cell, crew_cell, time_steps
    status, crew_cell, time_steps = take_step(matrix,aliens,crew_cell, bot_cell, time_steps)
    if status == "The bot was caught by the aliens!":
      return "The bot was caught by the aliens!", bot_cell, crew_cell, time_steps
    if matrix[bot_cell[0]][bot_cell[1]]['value']=='A' or matrix[bot_cell[0]][bot_cell[1]]['value']=='C*':
      return "The bot was caught by the aliens!", bot_cell, crew_cell, time_steps
  return "The bot has rescued a crewmate!", bot_cell, crew_cell, time_steps

**Bot 1**

In [ ]:
def breadth_first_search_bot_1(start_cell, matrix, aliens, crew_cell, vm_bfs_bot1, pm_bfs_bot1, time_steps):
  current_queue = [start_cell['self']]
  queue = [start_cell['self']]
  path = []                                                                     # entire sequence aka tree
  path1=[]                                                                      # shortest path
  temp=[]                                                                       # initialize path1
  while True:
    for x in current_queue:                                                     # iterate through current_queue
      if matrix[x[0]][x[1]]['value'] == 'C' or matrix[x[0]][x[1]]['value']=='C*':# checking if current element in queue is crewmate
        path.append(x)                                                          # adding crewmate cell to path
        path1.append(x)
        temp=x.copy()
        try:
          while pm_bfs_bot1[temp[0]][temp[1]]!=start_cell['self']:
          # while matrix[temp[0]][temp[1]]['prev']!=start_cell['self']:             # creating shortest path list in path1
            # path1.insert(0,matrix[temp[0]][temp[1]]['prev'])
            path1.insert(0,pm_bfs_bot1[temp[0]][temp[1]])
            # temp=matrix[temp[0]][temp[1]]['prev']
            temp=pm_bfs_bot1[temp[0]][temp[1]]
        except:
          print("ERROR VALUE IS",temp)
        path1.insert(0,start_cell['self'])
        return "found",path1,crew_cell, time_steps
      # matrix[x[0]][x[1]]['visited'] = 'true'
      vm_bfs_bot1[x[0]][x[1]] = 'true'                                   # marking each visited cell
      path.append(x)                                                            # appending current node to tree
      open_neighbors = detect_open_neighbors_bot_1(matrix[x[0]][x[1]])                # detecting neighbors of current node

      while len(open_neighbors)==0:                                             # if bot is blocked
        # print("From breadth_first_search_bot_1: bot is blocked, waiting 1 turn...")                                                 # VERY HELPFUL FOR DEBUG
        status, crew_cell, time_steps = take_step(matrix,aliens,crew_cell, bot_cell, time_steps)                                # wait while aliens move
        if status == "The bot was caught by the aliens!":
          return "The bot was caught by the aliens!", path1, crew_cell, time_steps
        open_neighbors = detect_open_neighbors(matrix[x[0]][x[1]])              # check if path still blocked
      for y in open_neighbors:
        # if y not in queue and matrix[y[0]][y[1]]['visited']=='false':
        if y not in queue and vm_bfs_bot1[y[0]][y[1]]=='false':

          # matrix[y[0]][y[1]]['prev']=x
          pm_bfs_bot1[y[0]][y[1]]=x                                          # asigning y as child of x
          queue.append(y)                                                       # adding y to queue
      queue.remove(x)
    current_queue = queue.copy()
    if matrix[bot_cell[0]][bot_cell[1]]['value']=='A' or matrix[bot_cell[0]][bot_cell[1]]['value']=='C*':
      return "The bot was caught by the aliens", path1, crew_cell, time_steps

In [ ]:
import random
D = 50
num_aliens = 50
# num_crew_res=0

results_array=[]

for _ in range(30):
  time_steps = 0
  num_crew_res=0

  visit_matrix_bot1 = []
  for i in range(D):
      row = []
      for j in range(D):
          row.append("false")
      visit_matrix_bot1.append(row)

  prev_matrix_bot1 = []
  for i in range(D):
      row = []
      for j in range(D):
          row.append([])
      prev_matrix_bot1.append(row)

  matrix, n = initialize_ship(D)
  matrix, aliens, crew_cell = explore_grid(n, matrix, num_aliens)
  bot_cell=create_bot(matrix,aliens)
  while (matrix[bot_cell[0]][bot_cell[1]]['value'] != 'A' and time_steps < 1000):
    print("initial ship:")
    # draw_ship(matrix)
    draw_ship_df(matrix)
    # for i in range(len(matrix)):
    #   for j in range(len(matrix)):
    #     print("VISITED VALUES ARE",matrix[i][j]['visited'])
    #     print("PREV VALUES ARE",matrix[i][j]['prev'])
    res, path, crew_cell, time_steps  = breadth_first_search_bot_1(matrix[bot_cell[0]][bot_cell[1]],matrix,aliens, crew_cell, visit_matrix_bot1, prev_matrix_bot1, time_steps)
    print("-----------------------------------------------------------------------------------------------------")
    print("crew member was "+res+" via path",path)
    if len(path)!=0:
      print("Now moving Bot to rescue Crewmate...")
    final_outcome, bot_cell, crew_cell, time_steps = move_bot(bot_cell,path,aliens,crew_cell, time_steps)
    print(final_outcome)
    if final_outcome=="The bot has rescued a crewmate!":
      num_crew_res+=1

      visit_matrix_bot1 = []
      for i in range(D):
        row = []
        for j in range(D):
          row.append("false")
        visit_matrix_bot1.append(row)

      prev_matrix_bot1 = []
      for i in range(D):
        row = []
        for j in range(D):
          row.append([])
        prev_matrix_bot1.append(row)
      # break
      continue
      # for i in range(len(matrix)):
      #   for j in range(len(matrix)):
      #     print("VISITED VALUES ARE",matrix[i][j]['visited'])
      #     print("PREV VALUES ARE",matrix[i][j]['prev'])
      # for i in range(len(matrix)):
      #   for j in range(len(matrix)):
      #     matrix[i][j]['visited']='false'
      #     matrix[i][j]['prev']=[]

    if res == "The bot was caught by the aliens" or final_outcome== 'The bot was caught by the aliens!' or final_outcome== 'The bot was caught by the aliens':
      print("NUMBER OF CREWMATES RESCUED",num_crew_res)
      results_array.append([num_crew_res, time_steps, "False"])
      break
  if time_steps == 1000:
    results_array.append([num_crew_res, time_steps,"True"])
  # draw_ship(matrix)
print(results_array)
results_csv=pd.DataFrame(results_array)
results_csv.to_csv("bot1_timed_size50aliens50.csv")

<h4>bot 2</h4>

In [ ]:
def breadth_first_search(start_cell, matrix, aliens, crew_cell, vm_bfs_bot2, pm_bfs_bot2, time_steps):
  current_queue = [start_cell['self']]
  queue = [start_cell['self']]
  path = []                                                                     # entire sequence aka tree
  path1=[]                                                                      # shortest path
  temp=[]                                                                       # initialize path1
  while True:
    for x in current_queue:                                                     # iterate through current_queue
      if matrix[x[0]][x[1]]['value'] == 'C' or matrix[x[0]][x[1]]['value']=='C*':# checking if current element in queue is crewmate
        path.append(x)                                                          # adding crewmate cell to path
        path1.append(x)
        temp=x.copy()
        # while matrix[temp[0]][temp[1]]['prev']!=start_cell['self']:
        while pm_bfs_bot2[temp[0]][temp[1]]!=start_cell['self']:             # creating shortest path list in path1
          # path1.insert(0,matrix[temp[0]][temp[1]]['prev'])
          path1.insert(0,pm_bfs_bot2[temp[0]][temp[1]])
          # temp=matrix[temp[0]][temp[1]]['prev']
          temp=pm_bfs_bot2[temp[0]][temp[1]]
        path1.insert(0,start_cell['self'])
        return "found",path1, crew_cell, vm_bfs_bot2, pm_bfs_bot2, time_steps
      vm_bfs_bot2[x[0]][x[1]] = 'true'
      # matrix[x[0]][x[1]]['visited'] = 'true'                                    # marking each visited cell
      path.append(x)                                                            # appending current node to tree
      open_neighbors = detect_open_neighbors(matrix[x[0]][x[1]])                # detecting neighbors of current node
      while len(open_neighbors)==0:                                             # if bot is blocked
        # print("From breadth_first_search: bot is blocked, waiting 1 turn...")                                                             # VERY HELPFUL FOR DEBUG
        status, crew_cell, time_steps = take_step(matrix,aliens,crew_cell, bot_cell, time_steps)                                # wait while aliens move
        if status == "The bot was caught by the aliens!":
          return "The bot was caught by the aliens!", path1, crew_cell, vm_bfs_bot2, pm_bfs_bot2, time_steps
        open_neighbors = detect_open_neighbors(matrix[x[0]][x[1]])              # check if path still blocked
      for y in open_neighbors:
        if y not in queue and vm_bfs_bot2[y[0]][y[1]]=='false':
        # if y not in queue and matrix[y[0]][y[1]]['visited']=='false':
          # matrix[y[0]][y[1]]['prev']=x
          pm_bfs_bot2[y[0]][y[1]]=x                                      # asigning y as child of x
          queue.append(y)                                                       # adding y to queue
      queue.remove(x)
    current_queue = queue.copy()
    if len(current_queue) == 0:
      # print("From breadth_first_search: not found error solution: bot is blocked, waiting 1 turn...")                                     # VERY HELPFUL FOR DEBUG
      status, crew_cell, time_steps = take_step(matrix,aliens,crew_cell, bot_cell, time_steps)
      if status == "The bot was caught by the aliens!":
        return "The bot was caught by the aliens!", path1, crew_cell, vm_bfs_bot2, pm_bfs_bot2, time_steps
      current_queue = [start_cell['self']]
      queue = [start_cell['self']]
      path = []                                                                     # entire sequence aka tree
      path1 = []                                                                      # shortest path
      temp = []
      for i in range(len(matrix)):
        for j in range(len(matrix)):
          pm_bfs_bot2[i][j] = []
          vm_bfs_bot2[i][j] = 'false'
          # matrix[i][j]['prev'] = []
          # matrix[i][j]['visited'] = 'false'

    if matrix[bot_cell[0]][bot_cell[1]]['value']=='A' or matrix[bot_cell[0]][bot_cell[1]]['value']=='C*':
      return "The bot was caught by the aliens", path1, crew_cell, vm_bfs_bot2, pm_bfs_bot2, time_steps

In [ ]:
def move_bot2(matrix,bot_cell,path,aliens,crew_start_cell,n, vm_movebot2, pm_movebot2, time_steps):
  crew_cell = crew_start_cell
  if matrix[bot_cell[0]][bot_cell[1]]['value']=='A' or matrix[bot_cell[0]][bot_cell[1]]['value']=='C*':
    return "The bot was caught by the aliens", bot_cell, crew_cell, vm_movebot2, pm_movebot2, time_steps
  for next in path[1::]:
    matrix[bot_cell[0]][bot_cell[1]]['value']=1
    matrix[next[0]][next[1]]['value']='B'
    bot_cell=next
    status, crew_cell, time_steps = take_step(matrix, aliens, crew_cell, bot_cell, time_steps)
    if status == "The bot was caught by the aliens!":
      return "The bot was caught by the aliens!", bot_cell, crew_cell, vm_movebot2, pm_movebot2, time_steps
    if matrix[bot_cell[0]][bot_cell[1]]['value']=='A' or matrix[bot_cell[0]][bot_cell[1]]['value']=='C*':
      return "The bot was caught by the aliens!", bot_cell, crew_cell, vm_movebot2, pm_movebot2, time_steps
    for i in range(n):
      for j in range(n):
        vm_movebot2[i][j]='false'
        # matrix[i][j]['visited']='false'
    res,path, crew_cell, vm_mb2, pm_mb2, time_steps =breadth_first_search(matrix[bot_cell[0]][bot_cell[1]],matrix,aliens, crew_cell, vm_movebot2, pm_movebot2, time_steps)
    if res == "The bot was caught by the aliens!":
      return "The bot was caught by the aliens!", bot_cell, crew_cell, vm_mb2, pm_mb2, time_steps
    # print("From move_bot2: following new path: ",res, path)                                        # VERY HELPFUL FOR DEBUG
  return "The bot has rescued a crewmate!", bot_cell, crew_cell, vm_mb2, pm_mb2, time_steps

In [ ]:
import random
D = 50
num_aliens = 5
num_crew_res = 0

results_array=[]

for _ in range(30):
  time_steps = 0
  num_crew_res=0

  visit_matrix_bot2 = []
  for i in range(D):
      row = []
      for j in range(D):
          row.append("false")
      visit_matrix_bot2.append(row)

  prev_matrix_bot2 = []
  for i in range(D):
      row = []
      for j in range(D):
          row.append([])
      prev_matrix_bot2.append(row)

  matrix, n = initialize_ship(D)
  matrix, aliens, crew_cell = explore_grid(n, matrix, num_aliens)
  bot_cell=create_bot(matrix,aliens)
  while (matrix[bot_cell[0]][bot_cell[1]]['value'] != 'A' and time_steps < 1000):
    print("initial ship:")
    draw_ship_df(matrix)
    res, path, crew_cell, pm_bot2, vm_bot2, time_steps  = breadth_first_search(matrix[bot_cell[0]][bot_cell[1]],matrix,aliens, crew_cell, visit_matrix_bot2, prev_matrix_bot2, time_steps)
    print("-----------------------------------------------------------------------------------------------------")
    print("crew member was "+res+" via path",path)
    if len(path)!=0:
      print("Now moving Bot to rescue Crewmate...")
    final_outcome, bot_cell, crew_cell, pm_bot2_temp, vm_bot2_temp, time_steps = move_bot2(matrix,bot_cell,path,aliens,crew_cell,n, pm_bot2, vm_bot2, time_steps)
    print(final_outcome)
    if final_outcome=="The bot has rescued a crewmate!":
      num_crew_res+=1

      visit_matrix_bot2 = []
      for i in range(D):
        row = []
        for j in range(D):
          row.append("false")
        visit_matrix_bot2.append(row)

      prev_matrix_bot2 = []
      for i in range(D):
        row = []
        for j in range(D):
          row.append([])
        prev_matrix_bot2.append(row)
      continue
      # break
    if res == "The bot was caught by the aliens" or final_outcome== 'The bot was caught by the aliens!' or final_outcome== 'The bot was caught by the aliens':
      print("NUMBER OF CREWMATES RESCUED",num_crew_res)
      results_array.append(num_crew_res, time_steps, "False")
      break
  if time_steps == 1000:
    results_array.append(num_crew_res, time_steps, "True")

  # draw_ship(matrix)

print(results_array)
results_csv=pd.DataFrame(results_array)
results_csv.to_csv("bot2_timed_size50_aliens5.csv")

<h4><b>Bot 3</b></h4>

In [ ]:
def bfs_bot2from3(start_cell, matrix, aliens, crew_cell,time_steps):
  current_queue = [start_cell['self']]
  queue = [start_cell['self']]
  path = []                                                                     # entire sequence aka tree
  path1=[]                                                                      # shortest path
  temp=[]

  vm23 = []
  for i in range(len(matrix)):
      row = []
      for j in range(len(matrix)):
          row.append("false")
      vm23.append(row)

  pm23 = []
  for i in range(len(matrix)):
      row = []
      for j in range(len(matrix)):
          row.append([])
      pm23.append(row)
                                                                         # initialize path1
  while True:
    for x in current_queue:                                                     # iterate through current_queue
      if matrix[x[0]][x[1]]['value'] == 'C' or matrix[x[0]][x[1]]['value']=='C*':# checking if current element in queue is crewmate
        path.append(x)                                                          # adding crewmate cell to path
        path1.append(x)
        temp=x.copy()
        # while matrix[temp[0]][temp[1]]['prev']!=start_cell['self']:
        while pm23[temp[0]][temp[1]]!=start_cell['self']:             # creating shortest path list in path1
          # path1.insert(0,matrix[temp[0]][temp[1]]['prev'])
          path1.insert(0,pm23[temp[0]][temp[1]])
          # temp=matrix[temp[0]][temp[1]]['prev']
          temp=pm23[temp[0]][temp[1]]
        path1.insert(0,start_cell['self'])
        return "Found",path1, vm23, pm23, time_steps
      vm23[x[0]][x[1]] = 'true'
      # matrix[x[0]][x[1]]['visited'] = 'true'                                    # marking each visited cell
      path.append(x)                                                            # appending current node to tree
      open_neighbors = detect_open_neighbors(matrix[x[0]][x[1]])                # detecting neighbors of current node
      # while len(open_neighbors)==0:                                             # if bot is blocked
      #   # print("From breadth_first_search: bot is blocked, waiting 1 turn...")                                                             # VERY HELPFUL FOR DEBUG
      #   status, crew_cell = take_step(matrix,aliens,crew_cell, bot_cell)                                # wait while aliens move
      #   if status == "The bot was caught by the aliens!":
      #     return "The bot was caught by the aliens!", path1, crew_cell, vm_bfs_bot2, pm_bfs_bot2
      #   open_neighbors = detect_open_neighbors(matrix[x[0]][x[1]])              # check if path still blocked
      for y in open_neighbors:
        if y not in queue and vm23[y[0]][y[1]]=='false':
        # if y not in queue and matrix[y[0]][y[1]]['visited']=='false':
          # matrix[y[0]][y[1]]['prev']=x
          pm23[y[0]][y[1]]=x                                      # asigning y as child of x
          queue.append(y)                                                       # adding y to queue
      queue.remove(x)
    current_queue = queue.copy()
    if len(current_queue) == 0:
      return "Not Found", path1, vm23, pm23, time_steps
      # print("From breadth_first_search: not found error solution: bot is blocked, waiting 1 turn...")                                     # VERY HELPFUL FOR DEBUG
      # # status, crew_cell = take_step(matrix,aliens,crew_cell, bot_cell)
      # if status == "The bot was caught by the aliens!":
      #   return "The bot was caught by the aliens!", path1, crew_cell, vm_bfs_bot2, pm_bfs_bot2

      # queue = [start_cell['self']]
      # path = []                                                                     # entire sequence aka tree
      # path1 = []                                                                      # shortest path
      # temp = []
      # for i in range(len(matrix)):
      #   for j in range(len(matrix)):
      #     pm_bfs_bot2[i][j] = []
      #     vm_bfs_bot2[i][j] = 'false'
          # matrix[i][j]['prev'] = []
          # matrix[i][j]['visited'] = 'false'

    # if matrix[bot_cell[0]][bot_cell[1]]['value']=='A' or matrix[bot_cell[0]][bot_cell[1]]['value']=='C*':
    #   return "The bot was caught by the aliens", path1, crew_cell, vm_bfs_bot2, pm_bfs_bot2

In [ ]:
#alien neighbors for bot 3
def detect_alien_neighbors(matrix, aliens):
  alien_neighbors=[]
  for alien_cell in aliens:
    for op_neigh in alien_cell['open_neighbors']:
      if op_neigh not in alien_neighbors:
        alien_neighbors.append(op_neigh)

  return alien_neighbors

In [ ]:
#BFS Bot 3
def breadth_first_search_bot_3(start_cell, matrix, aliens, crew_start_cell, vm_bfs_bot3, pm_bfs_bot3, time_steps):
  crew_cell =crew_start_cell
  current_queue = [start_cell['self']]
  queue = [start_cell['self']]
  path = []                                                                     # entire sequence aka tree
  path1=[]                                                                      # shortest path
  temp=[]                                                                       # initialize path1
  while True:
    for x in current_queue:                                                     # iterate through current_queue
      if matrix[x[0]][x[1]]['value'] == 'C' or matrix[x[0]][x[1]]['value']=='C*':# checking if current element in queue is crewmate
        path.append(x)                                                          # adding crewmate cell to path
        path1.append(x)
        temp=x.copy()
        while pm_bfs_bot3[temp[0]][temp[1]]!=start_cell['self']:
        # while matrix[temp[0]][temp[1]]['prev']!=start_cell['self']:             # creating shortest path list in path1
          # path1.insert(0,matrix[temp[0]][temp[1]]['prev'])
          path1.insert(0,pm_bfs_bot3[temp[0]][temp[1]])
          temp=pm_bfs_bot3[temp[0]][temp[1]]
          # temp=matrix[temp[0]][temp[1]]['prev']
        path1.insert(0,start_cell['self'])
        return "found",path1, crew_cell, vm_bfs_bot3, pm_bfs_bot3,time_steps
      # matrix[x[0]][x[1]]['visited'] = 'true'                                    # marking each visited cell
      vm_bfs_bot3[x[0]][x[1]] = 'true'
      path.append(x)                                                            # appending current node to tree
      open_neighbors = detect_open_neighbors(matrix[x[0]][x[1]])
      forbidden_open_neighbors=detect_alien_neighbors(matrix, aliens)               # detecting neighbors of current node
      open_neighbors = [x for x in open_neighbors if x not in forbidden_open_neighbors]
      while len(open_neighbors)==0:                                             # if bot is blocked
        # print("From breadth_first_search_bot_3: bot is blocked, waiting 1 turn...")                                        # VERY HELPFUL FOR DEBUG
        status, crew_start_cell,time_steps = take_step(matrix,aliens,crew_start_cell, bot_cell,time_steps)                                # wait while aliens move
        if status == "The bot was caught by the aliens!":
          return "The bot was caught by the aliens!", path1, crew_cell, vm_bfs_bot3, pm_bfs_bot3,time_steps
        open_neighbors = detect_open_neighbors(matrix[x[0]][x[1]])              # check if path still blocked
      for y in open_neighbors:
        if y not in queue and vm_bfs_bot3[y[0]][y[1]]=='false':
        # if y not in queue and matrix[y[0]][y[1]]['visited']=='false':
          # matrix[y[0]][y[1]]['prev']=x
          pm_bfs_bot3[y[0]][y[1]]=x                                        # asigning y as child of x
          queue.append(y)                                                       # adding y to queue
      queue.remove(x)
    current_queue = queue.copy()
    if len(current_queue) == 0:
      # print("From breadth_first_search_bot_3: path is blocked by alien, waiting 1 turn...")
                                             # VERY HELPFUL FOR DEBUG
      search_status, path_ret, vm23_return, pm23_return,time_steps = bfs_bot2from3(start_cell,matrix,aliens, crew_cell,time_steps)
      if (search_status=="Found"):
        print("PATH RETURNED FROM THE BFS 2 FUNCTION IS",path_ret)
        return "found", path_ret, crew_cell, vm23_return, pm23_return,time_steps

      status, crew_start_cell,time_steps = take_step(matrix,aliens,crew_start_cell, bot_cell,time_steps)
      if status == "The bot was caught by the aliens!":
        return "The bot was caught by the aliens!", path1, crew_cell, vm_bfs_bot3, pm_bfs_bot3,time_steps
      current_queue = [start_cell['self']]
      queue = [start_cell['self']]
      path = []                                                                     # entire sequence aka tree
      path1 = []                                                                      # shortest path
      temp = []
      for i in range(len(matrix)):
        for j in range(len(matrix)):
          pm_bfs_bot3[i][j] = []
          vm_bfs_bot3[i][j] = 'false'
          # matrix[i][j]['prev'] = []
          # matrix[i][j]['visited'] = 'false'

    if matrix[bot_cell[0]][bot_cell[1]]['value']=='A' or matrix[bot_cell[0]][bot_cell[1]]['value']=='C*':
      return "The bot was caught by the aliens!", path1, crew_cell, vm_bfs_bot3, pm_bfs_bot3,time_steps

In [ ]:
def move_bot3(matrix,bot_cell,path,aliens,crew_start_cell,n, vm_bot3_mb3, pm_bot3_mb3,time_steps):
  crew_cell = crew_start_cell

  if matrix[bot_cell[0]][bot_cell[1]]['value']=='A' or matrix[bot_cell[0]][bot_cell[1]]['value']=='C*':
    return "The bot was caught by the aliens", bot_cell, crew_cell, vm_bot3_mb3, pm_bot3_mb3,time_steps
  for next in path[1::]:
    matrix[bot_cell[0]][bot_cell[1]]['value']=1
    matrix[next[0]][next[1]]['value']='B'
    bot_cell=next
    status, crew_cell,time_steps = take_step(matrix,aliens,crew_cell, bot_cell,time_steps)
    if status == "The bot was caught by the aliens!":
      return "The bot was caught by the aliens!", bot_cell, crew_cell, vm_bot3_mb3, pm_bot3_mb3,time_steps
    if matrix[bot_cell[0]][bot_cell[1]]['value']=='A' or matrix[bot_cell[0]][bot_cell[1]]['value']=='C*':
      return "The bot was caught by the aliens!", bot_cell, crew_cell, vm_bot3_mb3, pm_bot3_mb3,time_steps
    for i in range(n):
      for j in range(n):
        # matrix[i][j]['visited']='false'
        vm_bot3_mb3[i][j]='false'
    res, path, crew_cell, vm_return_bot3, pm_return_bot3,time_steps=breadth_first_search_bot_3(matrix[bot_cell[0]][bot_cell[1]],matrix,aliens, crew_cell, vm_bot3_mb3, pm_bot3_mb3,time_steps)
    if res == "The bot was caught by the aliens!":
      return "The bot was caught by the aliens!", bot_cell, crew_cell, vm_return_bot3, pm_return_bot3,time_steps
    # print("From move_bot3: following new path: ",res, path)                                         # VERY HELPFUL FOR DEBUG
  return "The bot has rescued a crewmate!", bot_cell, crew_cell, vm_return_bot3, pm_return_bot3,time_steps

In [ ]:
# 5,7,10,12, 15, 17,20
import random
D = 50
num_aliens = 5
num_crew_res=0

results_array=[]

for _ in range(30):
  num_crew_res=0
  time_steps=0
  visit_matrix_bot3 = []
  for i in range(D):
      row = []
      for j in range(D):
          row.append("false")
      visit_matrix_bot3.append(row)

  prev_matrix_bot3 = []
  for i in range(D):
      row = []
      for j in range(D):
          row.append([])
      prev_matrix_bot3.append(row)

  matrix, n = initialize_ship(D)
  matrix, aliens, crew_cell = explore_grid(n, matrix, num_aliens)
  bot_cell=create_bot(matrix,aliens)
  crew_memebers_rescued = 0
  # for _ in range(2):
  while (matrix[bot_cell[0]][bot_cell[1]]['value'] != 'A' and time_steps<1000):
    print("initial ship:")
    draw_ship_df(matrix)
    res,path, crew_cell, vm3, pm3,time_steps=breadth_first_search_bot_3(matrix[bot_cell[0]][bot_cell[1]],matrix,aliens, crew_cell, visit_matrix_bot3, prev_matrix_bot3,time_steps)
    print("-----------------------------------------------------------------------------------------------------")
    print("crew member was "+res+" via path",path)
    if len(path)!=0:
      print("Now moving Bot to rescue Crewmate...")
    final_outcome, bot_cell, crew_cell, vm_return3, pm_return3,time_steps = move_bot3(matrix,bot_cell,path,aliens,crew_cell,n, vm3, pm3,time_steps)
    print(final_outcome)
    if final_outcome=="The bot has rescued a crewmate!":
      num_crew_res+=1

      visit_matrix_bot3 = []
      for i in range(D):
        row = []
        for j in range(D):
          row.append("false")
        visit_matrix_bot3.append(row)

      prev_matrix_bot3 = []
      for i in range(D):
        row = []
        for j in range(D):
          row.append([])
        prev_matrix_bot3.append(row)
      continue
      # break
    if res == "The bot was caught by the aliens" or final_outcome== 'The bot was caught by the aliens!' or final_outcome== 'The bot was caught by the aliens':
      print("NUMBER OF CREWMATES RESCUED",num_crew_res)
      results_array.append([num_crew_res, time_steps, "False"])
      break
    if time_steps == 1000:
      results_array.append([num_crew_res, time_steps, "True"])

print(results_array)
results_csv=pd.DataFrame(results_array)
results_csv.to_csv("bot3_timed_size50_aliens5.csv")
# print(final_outcome)
# print("Crew Memebers Rescued: ", crew_memebers_rescued)
# print("Final Ship:")
# draw_ship_df(matrix)

In [ ]:
 import numpy as np


np.mean([4, 2, 3, 5, 0, 0, 1, 3, 0, 0, 0, 0, 0, 0, 9, 0, 1, 1, 0, 0, 1, 0, 4, 0, 1, 0, 0, 0, 0])

In [ ]:
[[5, 3, 4, 4, 1, 3, 31, 0, 9, 3, 13, 2, 3, 1, 3, 8, 3, 6, 1, 7, 9, 2, 2, 1, 16, 0, 1, 5, 16],
 ]
[[50,15,5.586206896551724]]

<h4><b>Bot 4</b></h4>

In [ ]:
def move_bot4(matrix,bot_cell,path,aliens,crew_start_cell,n, vm_bot4, pm_bot4,time_steps):
  crew_cell = crew_start_cell
  if matrix[bot_cell[0]][bot_cell[1]]['value']=='A' or matrix[bot_cell[0]][bot_cell[1]]['value']=='C*':
    return "The bot was caught by the aliens", bot_cell, vm_bot4, pm_bot4, crew_cell,time_steps
  for next in path[1::]:
    # print("From move_bot4: Bot moving from: ", bot_cell, "to ", next)                                                     # VERY HELPFUL FOR DEBUG
    matrix[bot_cell[0]][bot_cell[1]]['value']=1
    matrix[next[0]][next[1]]['value']='B'
    bot_cell=next
    # print("From move_bot4: Taking Step")                                                                                  # VERY HELPFUL FOR DEBUG
    # print("From move_bot4 crew cell: ", crew_cell)                                                                        # VERY HELPFUL FOR DEBUG
    status, crew_cell,time_steps = take_step(matrix,aliens, crew_cell, bot_cell,time_steps)
    # print("From move_bot4 new crew cell: ", crew_cell)                                                                    # VERY HELPFUL FOR DEBUG
    if status == "The bot was caught by the aliens!":
      # print("From move_bot4: take step has detected bot being caught by aliens")                                          # VERY HELPFUL FOR DEBUG
      return "The bot was caught by the aliens!", bot_cell, vm_bot4, pm_bot4, crew_cell,time_steps
    if matrix[bot_cell[0]][bot_cell[1]]['value']=='A' or matrix[bot_cell[0]][bot_cell[1]]['value']=='C*':
      return "The bot was caught by the aliens!", bot_cell, vm_bot4, pm_bot4, crew_cell,time_steps
    for i in range(n):
      for j in range(n):
        vm_bot4[i][j]='false'
    res,path, vm_return, pm_return,time_steps=A_star_bot_4(matrix[bot_cell[0]][bot_cell[1]],matrix,aliens, vm_bot4, pm_bot4, crew_cell,time_steps)
    if res == "The bot was caught by the aliens!":
      return "The bot was caught by the aliens!", bot_cell, vm_return, pm_return, crew_cell,time_steps
    # print("From move_bot4: following new path: ",res, path)                                                               # VERY HELPFUL FOR DEBUG
  return "The bot has rescued a crewmate!", bot_cell, vm_return, pm_return, crew_cell,time_steps

In [ ]:
# def heuristic_bot_4(start_cell, matrix, visit_matrix2, prev_matrix2):
#   current_queue = [start_cell['self']]
#   queue = [start_cell['self']]
#   path = []                                                                     # entire sequence aka tree
#   path1=[]                                                                      # shortest path
#   temp=[]
#   hval=0                                                                  # initialize path1
#   while True:
#     if start_cell['value'] == 'C' or start_cell['value']=='C*':
#       return hval
#     for x in current_queue:                                                     # iterate through current_queue
#       if matrix[x[0]][x[1]]['value'] == 'C' or matrix[x[0]][x[1]]['value']=='C*':# checking if current element in queue is crewmate
#         path.append(x)                                                          # adding crewmate cell to path
#         path1.append(x)
#         temp=x

#         while prev_matrix2[temp[0]][temp[1]]!=start_cell['self']:             # creating shortest path list in path1
#           path1.insert(0,prev_matrix2[temp[0]][temp[1]])
#           hval+=1
#           temp=prev_matrix2[temp[0]][temp[1]]
#         path1.insert(0,start_cell['self'])
#         return hval+1

#       visit_matrix2[x[0]][x[1]] = 'true'                                  # marking each visited cell
#       path.append(x)                                                            # appending current node to tree
#       open_neighbors = detect_open_neighbors_bot_1(matrix[x[0]][x[1]])                # detecting neighbors of current node
#       for y in open_neighbors:
#         if y not in queue and visit_matrix2[y[0]][y[1]]=='false':
#           prev_matrix2[y[0]][y[1]]=x                                          # asigning y as child of x
#           queue.append(y)                                                       # adding y to queue
#       queue.remove(x)

#     current_queue = queue.copy()

In [ ]:
# def find_heuristic(index,matrix):
#   matrix1=matrix.copy()

#   visit_matrix2 = []
#   for i in range(len(matrix1)):
#       row = []
#       for j in range(len(matrix1)):
#           row.append("false")
#       visit_matrix2.append(row)

#   prev_matrix2 = []
#   for i in range(len(matrix1)):
#       row = []
#       for j in range(len(matrix1)):
#           row.append([])
#       prev_matrix2.append(row)

#   for i in range(len(matrix1)):
#     for j in range(len(matrix1)):
#       prev_matrix2[i][j] = []
#       visit_matrix2[i][j] = 'false'

#   val=heuristic_bot_4(matrix1[index[0]][index[1]],matrix1, visit_matrix2, prev_matrix2)

#   return (val,index)

In [ ]:
def heuristic_bot_4(curr,crew,aliens):
  # print("CURRENT CELL CELL IS",curr)
  # print("CREWMATE CELL IS",crew)
  goal_hval=abs(curr[0]-crew[0])+abs(curr[1]-crew[1])                                         # manhatta disstance crew memeber
  # print("GOAL HVAL IS",goal_hval)
  alien_min1=abs(curr[0]-aliens[0]['self'][0])+abs(curr[1]-aliens[0]['self'][1])              #intialize mh distance alien1
  alien_min2=alien_min1                                                                       # initialize mh distance alien2
  alien_min1_pos=[aliens[0]['self'][0],aliens[0]['self'][1]]
  alien_min2_pos=[]
  aliens_ordered_array=[]
  for i,alien in enumerate(aliens):
    d1=abs(curr[0]-aliens[i]['self'][0])+abs(curr[1]-aliens[i]['self'][1])                    # calculates mh distance to aliens
    v1=[aliens[i]['self'][0],aliens[i]['self'][1]]                                            # alien coords
    # print("DISTANCE OF CURRENT CELL",curr,"FROM",v1,"IS",d1)
    aliens_ordered_array.append([d1,v1])                                                      # array of alien mh distance and coords
    # if abs(curr[0]-aliens[i]['self'][0])+abs(curr[1]-aliens[i]['self'][1]) < alien_min1:
    #   alien_min2=alien_min1
    #   alien_min2_pos=alien_min1_pos
    #   alien_min1=abs(curr[0]-aliens[i]['self'][0])+abs(curr[1]-aliens[i]['self'][1])
    #   alien_min1_pos=[aliens[i]['self'][0],aliens[i]['self'][1]]
  sorted_alien_lst = sorted(aliens_ordered_array, key=lambda x: x[0])                         # sorting array
  alien_min1_pos=sorted_alien_lst[0][1]
  alien_min1=sorted_alien_lst[0][0]
  alien_min2_pos=sorted_alien_lst[1][1]
  alien_min2=sorted_alien_lst[1][0]
  # print("ALIEN MIN VAL 1 IS AT",alien_min1_pos,"AND THE DISTANCE IS",alien_min1)
  # print("ALIEN MIN VAL 2 IS AT",alien_min2_pos,"AND THE DISTANCE IS",alien_min2)
  alien_hval=(alien_min1+alien_min2)//2                                                       # averaging aliens distance
  # print("ALIEN HVAL IS",alien_hval)
  hval=2*goal_hval+8*alien_hval                                                               # heuristic using weights
  # print("FINAL HVAL IS",hval)
  # print ("HEURISTIC RETURNED FOR",curr,"IS",hval)
  return (hval,curr)

In [ ]:
def check_if_visited(vm_check, a,b):
  if vm_check[a][b]=='true':
    return "true"
  else:
    return "false"

In [ ]:
def draw_visited_matrix(vm_draw):
  for i in range(len(vm_draw)):
    for j in range(len(vm_draw)):
      if vm_draw[i][j]=='true':
        print(i,j,vm_draw[i][j])

In [ ]:
def A_star_bot_4(start_cell, matrix1, aliens, vm_astar, pm_astar, crew_start_cell,time_steps):
  crew_cell = crew_start_cell
  current_queue = [start_cell['self']]
  queue = [start_cell['self']]
  path = []                                                                     # entire sequence aka tree
  path1=[]                                                                      # shortest path
  temp=[]
  pm_astar[start_cell['self'][0]][start_cell['self'][1]]=start_cell['self']                                         # initialize path1

  while True:
    for x in current_queue:
      if matrix1[x[0]][x[1]]['value'] == 'C' or matrix1[x[0]][x[1]]['value']=='C*':# checking if current element in queue is crewmate
        path.append(x)                                                          # adding crewmate cell to path
        path1.append(x)
        temp=x

        while pm_astar[temp[0]][temp[1]]!=start_cell['self']:             # creating shortest path list in path1
            path1.insert(0,pm_astar[temp[0]][temp[1]])
            temp=pm_astar[temp[0]][temp[1]]
        path1.insert(0,start_cell['self'])
        # print("From A_star_bot_4: Current bot position", start_cell['self'])                                        # VERY HELPFUL FOR DEBUG
        # print("From A_star_bot_4: PATH FOUND",path1)                                                                # VERY HELPFUL FOR DEBUG
        return "found",path1, vm_astar, pm_astar,time_steps

      vm_astar[x[0]][x[1]]= 'true'                                    # marking each visited cell
      path.append(x)                                                            # appending current node to tree
      open_neighbors = detect_open_neighbors(matrix1[x[0]][x[1]])                # detecting neighbors of current node

      on1=[]
      for ele in open_neighbors:
        on1.append(heuristic_bot_4(ele,crew_cell,aliens))
      on1.sort()

      on2=[]
      for ele in on1:
        temp1=ele[1]
        on2.append(temp1)

      while len(open_neighbors)==0:                                             # if bot is blocked
        # print("From A_star_bot_4: Current bot position", start_cell['self'])                                        # VERY HELPFUL FOR DEBUG
        # print("From A_star_bot_4: path is blocked, waiting 1 turn...")                                              # VERY HELPFUL FOR DEBUG
        # print("From A_star_bot_4: taking_step")                                                                     # VERY HELPFUL FOR DEBUG
        status, crew_cel,time_steps = take_step(matrix1,aliens, crew_cell, bot_cell,time_steps)                                # wait while aliens move

        if status == "The bot was caught by the aliens!":
          # print("From A_star_bot_4: take_step has detected the bot was caught by aliens")                                        # VERY HELPFUL FOR DEBUG
          return "The bot was caught by the aliens!", path1, vm_astar, pm_astar,time_steps

        open_neighbors = detect_open_neighbors(matrix1[x[0]][x[1]])              # check if path still blocked

      for y in on2:
        if (y==start_cell['self']):
          continue

        if y not in queue and check_if_visited(vm_astar, y[0],y[1])=='false':
          pm_astar[y[0]][y[1]]=x
          queue.append(y)                                                       # adding y to queue
      queue.remove(x)
      current_queue = queue.copy()

      if len(current_queue) == 0:
        # print("From A_star_bot_4: Current bot position", start_cell['self'])                                                            # VERY HELPFUL FOR DEBUG
        # print("From A_star_bot_4: not found error solution: bot is blocked, waiting 1 turn...")                                        # VERY HELPFUL FOR DEBUG
        # print("From A_star_bot_4: taking step")                                                                                         # VERY HELPFUL FOR DEBUG
        status, crew_cell,time_steps = take_step(matrix1,aliens,crew_cell, bot_cell,time_steps)
        if status == "The bot was caught by the aliens!":
          return "The bot was caught by the aliens!", path1, vm_astar, pm_astar,time_steps

        current_queue = [start_cell['self']]
        queue = [start_cell['self']]
        path = []                                                                     # entire sequence aka tree
        path1 = []                                                                      # shortest path
        temp = []
        for i in range(len(matrix1)):
          for j in range(len(matrix1)):
            pm_astar[i][j] = []
            vm_astar[i][j] = 'false'

        A_star_bot_4(start_cell, matrix1, aliens, vm_astar, pm_astar, crew_cell,time_steps)


    if matrix1[bot_cell[0]][bot_cell[1]]['value']=='A' or matrix1[bot_cell[0]][bot_cell[1]]['value']=='C*':
      return "The bot was caught by the aliens", path1, vm_astar, pm_astar,time_steps

In [ ]:
# import random
# D = 10
# num_aliens = 10

# visit_matrix = []
# for i in range(D):
#     row = []
#     for j in range(D):
#         row.append("false")
#     visit_matrix.append(row)

# prev_matrix = []
# for i in range(D):
#     row = []
#     for j in range(D):
#         row.append([])
#     prev_matrix.append(row)

# matrix, n = initialize_ship(D)
# matrix, aliens, crew_cell = explore_grid(n, matrix, num_aliens)
# bot_cell=create_bot(matrix,aliens)
# matrix[bot_cell[0]][bot_cell[1]]['prev']=[-1,-1]
# while (matrix[bot_cell[0]][bot_cell[1]]['value'] != 'A'):
#   print("initial ship:")
#   draw_ship(matrix)
#   res,path,vm2,pm2=A_star_bot_4(matrix[bot_cell[0]][bot_cell[1]],matrix,aliens, visit_matrix, prev_matrix, crew_start_cell)
#   print("-----------------------------------------------------------------------------------------------------")
#   print("crew member was "+res+" via path",path)
#   if len(path)!=0:
#     print("Now moving Bot to rescue Crewmate...")
#   final_outcome, bot_cell, vmatrix, pmatrix, crew_cell = move_bot4(matrix,bot_cell,path,aliens, crew_cell,n,vm2,pm2)
#   print(final_outcome)
#   if final_outcome=="The bot has rescued a crewmate!":
#     break
#   if res == "The bot was caught by the aliens":
#     break
#   draw_ship(matrix)

# [[5, 5], [5, 6], [4, 6], [3, 6], [3, 5], [3, 4], [2, 4], [1, 4], [1, 3], [0, 3]]

In [ ]:
import random

D = 50
num_aliens = 15
results_array=[]
for _ in range(30):
  visit_matrix = []
  for i in range(D):
      row = []
      for j in range(D):
          row.append("false")
      visit_matrix.append(row)

  prev_matrix = []
  for i in range(D):
      row = []
      for j in range(D):
          row.append([])
      prev_matrix.append(row)

  new_matrix = True
  if new_matrix:
    matrix, n = initialize_ship(D)
    matrix, aliens, crew_cell = explore_grid(n, matrix, num_aliens)

  bot_cell=create_bot(matrix,aliens)
  matrix[bot_cell[0]][bot_cell[1]]['prev']=[-1,-1]
  time_steps=0
  crew_members_rescued = 0

  iterations = 0
  #draw_ship(matrix)
  #for _ in range(2):
  while (matrix[bot_cell[0]][bot_cell[1]]['value'] != 'A' and time_steps<1000):
      print("initial ship:")
      # draw_ship(matrix)
      draw_ship_df(matrix)
      res,path,vm2,pm2,time_steps=A_star_bot_4(matrix[bot_cell[0]][bot_cell[1]],matrix,aliens, visit_matrix, prev_matrix, crew_cell,time_steps)
      print("-----------------------------------------------------------------------------------------------------")                                        # VERY HELPFUL FOR DEBUG
      # print("From current code: crew member was "+res+" via path",path)                                         # VERY HELPFUL FOR DEBUG
      # if len(path)!=0:
        # print("From current code: Now moving Bot to rescue Crewmate...")                                        # VERY HELPFUL FOR DEBUG
      final_outcome, bot_cell, vmatrix, pmatrix, crew_cell,time_steps = move_bot4(matrix,bot_cell,path,aliens,crew_cell,n,vm2,pm2,time_steps)
      # print("From current code: ", final_outcome)                                        # VERY HELPFUL FOR DEBUG
      iterations = iterations + 1                                                                                 # for some reason these dont run????

      if final_outcome=="The bot has rescued a crewmate!":
        crew_members_rescued = crew_members_rescued + 1
        visit_matrix = []
        for i in range(D):
          row = []
          for j in range(D):
            row.append("false")
          visit_matrix.append(row)

        prev_matrix = []
        for i in range(D):
          row = []
          for j in range(D):
            row.append([])
          prev_matrix.append(row)
      if res == "The bot was caught by the aliens!" or final_outcome == "The bot was caught by the aliens!" or final_outcome== 'The bot was caught by the aliens':
        final_outcome = "The bot was caught by the aliens"
        print("NUMBER OF CREWMATES RESCUED",crew_members_rescued)
        results_array.append([crew_members_rescued, time_steps, "False"])
        results_csv=pd.DataFrame(results_array)
        results_csv.to_csv("bot4_timed_size50_aliens15.csv")
        break
      if time_steps == 1000:
        results_array.append([crew_members_rescued, time_steps, "True"])
      print(final_outcome)
    #draw_ship(matrix)                                                                                            # for some reason these dont run????
print("the result array is: ",results_array)
results_csv=pd.DataFrame(results_array)
results_csv.to_csv("bot4_timed_size50_aliens15.csv")


#print("From current code: ", final_outcome)
#print("From current code: Crew Memebers Rescued: ",crew_memebers_rescued)

In [ ]:
results_csv=pd.DataFrame(results_array)
results_csv.to_csv("bot4_timed_size50_aliens5.csv")

NameError: ignored